In [1]:
import json
import openai

In [2]:
client = openai.Client()

In [4]:
def saudacao_por_periodo(hora):
    if 5 <= hora < 12:
        return json.dumps({"saudacao": "bom dia"})
    elif 12 <= hora < 18:
        return json.dumps({"saudacao": "boa tarde"})
    elif 18 <= hora < 22:
        return json.dumps({"saudacao": "boa noite"})
    else:
        return json.dumps({"saudacao": "boa madrugada"})

In [28]:
tools = [
    {
        "type" : "function",
        "function" : {
            "name": "saudacao_por_periodo",
            "description" : "retorna uma saudacao baseada na hora do dia",
            "parameters": {
                "type": "object",
                "properties": {
                    "hora": {
                        "type": "integer",
                        "description": "a hora do dia em formato de 24h"
                    },
                },
                "required": ["hora"]
            }
        }
    }
]

In [29]:
funcao_disponivel ={
    "saudacao_por_periodo" : saudacao_por_periodo
}

In [30]:
mensagens = [{"role": "user", "content": "Qual a saudacao o modelo me d'a se for 10h?"}]

In [31]:
resposta = client.chat.completions.create(
    model = "gpt-3.5-turbo-0125",
    messages = mensagens,
    tools = tools,
    tool_choice="auto"
)

In [32]:
mensagem_resp = resposta.choices[0].message
mensagem_resp

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_xUiAyuq1qb0mucdffNqWhxRK', function=Function(arguments='{"hora":10}', name='saudacao_por_periodo'), type='function')])

In [33]:
tool_calls = mensagem_resp.tool_calls
tool_calls

[ChatCompletionMessageToolCall(id='call_xUiAyuq1qb0mucdffNqWhxRK', function=Function(arguments='{"hora":10}', name='saudacao_por_periodo'), type='function')]

In [34]:
if tool_calls:
    mensagens.append(mensagem_resp)
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = funcao_disponivel[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            hora = function_args.get("hora")
        )

        mensagens.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response
            }
        )

    segunda_resposta = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=mensagens,
    )


In [35]:
mensgem_resposta = segunda_resposta.choices[0].message
mensgem_resposta

ChatCompletionMessage(content='A saudação que o modelo me dá às 10h é "bom dia".', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)